1. Introduction
    * 1.1  Problem definition
    * 1.2  Datasets
2. Environment
    * 2.1  Packages
    * 2.2  Parameters
    * 2.3  Functions
    * 2.4  Data
3. Wrangling
    * 3.1  Typecasting
    * 3.2  Renaming
    * 3.3  Sorting
4. Conclusion
    * 4.1  Results
    * 4.2  Next steps

## 2. Environment

### 2.1 Packages

In [1]:
import os
import pandas as pd
import numpy as np

# own utils
from data_eng_utils import calc_df_size

# printing
from beeprint import pp
import pprint

### 2.2 Parameters

In [2]:
# https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory
os.listdir()

['.ipynb_checkpoints',
 'Aggregated_Metrics_By_Country_And_Subscriber_Status.csv',
 'Aggregated_Metrics_By_Video.csv',
 'All_Comments_Final.csv',
 'data_eng_utils.py',
 'kengee_youtube_data.ipynb',
 'Video_Performance_Over_Time.csv',
 '__pycache__']

In [3]:
ROOT_DIR = './'

COUNTRY_SUB_METRICS = 'Aggregated_Metrics_By_Country_And_Subscriber_Status.csv'
VIDEO_METRICS = 'Aggregated_Metrics_By_Video.csv'
ALL_COMMENTS = 'All_Comments_Final.csv'
VIDEO_PERFORMANCE = 'Video_Performance_Over_Time.csv'

### 2.4 Data

Checking initial memory consumption prior to cleaning

In [42]:
# Reading data
country_metrics = pd.read_csv(COUNTRY_SUB_METRICS, encoding='utf-8')
video_metrics = pd.read_csv(VIDEO_METRICS, encoding='utf-8')
comments = pd.read_csv(ALL_COMMENTS, encoding='utf-8')
video_performance = pd.read_csv(VIDEO_PERFORMANCE, encoding='utf-8')

In [43]:
def calc_df_size(*args):
    tot_size = 0
    for df in args:
        tot_size += df.memory_usage().sum()
    return tot_size/1000000

In [44]:
size = calc_df_size(country_metrics, video_metrics, comments, video_performance)
print(f'Size is {size} MB')

Size is 19.38398 MB


## 3. Wrangling
- process of transforming and mapping data from one "raw" data form into another format with the intent of making it more appropriate and valuable for a variety of downstream purposes such as analytics. 

In [7]:
# Analyis of country_metrics 
country_metrics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55292 entries, 0 to 55291
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Video Title                 55292 non-null  object 
 1   External Video ID           55292 non-null  object 
 2   Video Length                55292 non-null  int64  
 3   Thumbnail link              55292 non-null  object 
 4   Country Code                54906 non-null  object 
 5   Is Subscribed               55292 non-null  bool   
 6   Views                       55292 non-null  int64  
 7   Video Likes Added           55292 non-null  int64  
 8   Video Dislikes Added        55292 non-null  int64  
 9   Video Likes Removed         55292 non-null  int64  
 10  User Subscriptions Added    55292 non-null  int64  
 11  User Subscriptions Removed  55292 non-null  int64  
 12  Average View Percentage     53854 non-null  float64
 13  Average Watch Time          538

In [8]:
country_metrics.head(2)

,Video Title,External Video ID,Video Length,Thumbnail link,Country Code,Is Subscribed,Views,Video Likes Added,Video Dislikes Added,Video Likes Removed,User Subscriptions Added,User Subscriptions Removed,Average View Percentage,Average Watch Time,User Comments Added
0,🌶 Hot Topics in Tech: Data Science Explained #...,OtqQYqRNDGI,59,https://i.ytimg.com/vi/OtqQYqRNDGI/hqdefault.jpg,HK,True,23,1,0,0,2,0,0.67187,39.640348,0
1,🌶 Hot Topics in Tech: Data Science Explained #...,OtqQYqRNDGI,59,https://i.ytimg.com/vi/OtqQYqRNDGI/hqdefault.jpg,ME,True,3,0,0,0,0,0,0.49887,29.433333,0


In [9]:
COUNTRY_METRICS_SCHEMA = {
    "Video Title": {
        "title": "title",
        "data_type": str
    },
    "External Video ID": {
        "title": "video_id",
        "data_type": str
    },
    "Video Length": {
        "title": "length_sec",
        "data_type": int
    },
    "Thumbnail link": {
        "title": "thumbnail",
        "data_type": str
    },
    "Country Code": {
        "title": "coutry",
        "data_type": str
    },
    "Is Subscribed": {
        "title": "subscribed",
        "data_type": bool
    },
    "Views": {
        "title": "views",
        "data_type": int
    },
    "Video Likes Added": {
        "title": "likes_added",
        "data_type": int
    },
    "Video Dislikes Added": {
        "title": "dislikes_added",
        "data_type": int
    },
    "Video Likes Removed": {
        "title": "likes_removed",
        "data_type": int
    },
    "User Subscriptions Added": {
        "title": "subscription_added",
        "data_type": int
    },
    "User Subscriptions Removed": {
        "title": "subscription_removed",
        "data_type": int
    },
    "Average View Percentage": {
        "title": "avg_views_percent",
        "data_type": float
    },
    "Average Watch Time": {
        "title": "avg_watch_time",
        "data_type": float
    },
    "User Comments Added": {
        "title": "comments",
        "data_type": int
    }
}

In [10]:
def schema_output(df):
    for col in df.columns:
        print(f'"{col}":', '{')
        print("\t\"title\": \"\",")
        print("\t\"data_type\": \"\"")
        print("}", end='')
        if df.columns[len(df.columns) - 1] != col:
            print(',', end='')
        print()

In [11]:
# Analysis of comments
comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10240 entries, 0 to 10239
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Comments     10239 non-null  object
 1   Comment_ID   10240 non-null  object
 2   Reply_Count  10240 non-null  int64 
 3   Like_Count   10240 non-null  int64 
 4   Date         10240 non-null  object
 5   VidId        10240 non-null  object
 6   user_ID      10240 non-null  object
dtypes: int64(2), object(5)
memory usage: 560.1+ KB


In [12]:
comments.head(2)

,Comments,Comment_ID,Reply_Count,Like_Count,Date,VidId,user_ID
0,Thanks for this video Ken.\n\nI decided to go ...,UgxFZTIzC4UPyhhX_PZ4AaABAg,0,0,2022-01-22T08:13:29Z,xpIFS6jZbe8,user_981
1,Hello ken jee!!! I'm doing a graduation on Com...,Ugx3IYuAufhb932GYQx4AaABAg,0,0,2022-01-22T01:09:46Z,xpIFS6jZbe8,user_213


In [13]:
COMMENTS_SCHEMA = {
    "Comments": {
        "title": "comment",
        "data_type": str
    },
    "Comment_ID": {
        "title": "comment_id",
        "data_type": str
    },
    "Reply_Count": {
        "title": "reply_count",
        "data_type": int
    },
    "Like_Count": {
        "title": "like_count",
        "data_type": int
    },
    "Date": {
        "title": "date",
        "data_type": "datetime"
    },
    "VidId": {
        "title": "video_id",
        "data_type": str
    },
    "user_ID": {
        "title": "user_id",
        "data_type": str
    }
}

In [14]:
# Analysis of Video Performance
video_performance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111857 entries, 0 to 111856
Data columns (total 14 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Date                        111857 non-null  object 
 1   Video Title                 111857 non-null  object 
 2   External Video ID           111857 non-null  object 
 3   Video Length                111857 non-null  int64  
 4   Thumbnail link              111857 non-null  object 
 5   Views                       111857 non-null  int64  
 6   Video Likes Added           111857 non-null  int64  
 7   Video Dislikes Added        111857 non-null  int64  
 8   Video Likes Removed         111857 non-null  int64  
 9   User Subscriptions Added    111857 non-null  int64  
 10  User Subscriptions Removed  111857 non-null  int64  
 11  Average View Percentage     110510 non-null  float64
 12  Average Watch Time          110510 non-null  float64
 13  User Comments 

In [15]:
video_performance.head(2)

,Date,Video Title,External Video ID,Video Length,Thumbnail link,Views,Video Likes Added,Video Dislikes Added,Video Likes Removed,User Subscriptions Added,User Subscriptions Removed,Average View Percentage,Average Watch Time,User Comments Added
0,19 Jan 2022,Kaggle Project From Scratch - Part 2 (Explorat...,KQ80oD_boBM,2191,https://i.ytimg.com/vi/KQ80oD_boBM/hqdefault.jpg,13,0,0,0,0,0,0.069055,151.300154,0
1,19 Jan 2022,Welcome To My Channel | Ken Jee | Data Science,smeFkHwnM_k,51,https://i.ytimg.com/vi/smeFkHwnM_k/hqdefault.jpg,2,0,0,0,1,0,0.471255,24.034000,0


In [16]:
VIDEO_PERFORMANCE_SCHEMA = {
    "Date": {
        "title": "date",
        "data_type": "datetime"
    },
    "Video Title": {
        "title": "title",
        "data_type": str
    },
    "External Video ID": {
        "title": "video_id",
        "data_type": str
    },
    "Video Length": {
        "title": "length_sec",
        "data_type": int
    },
    "Thumbnail link": {
        "title": "thumbnail",
        "data_type": str
    },
    "Views": {
        "title": "views",
        "data_type": int
    },
    "Video Likes Added": {
        "title": "likes_added",
        "data_type": int
    },
    "Video Dislikes Added": {
        "title": "dislikes_added",
        "data_type": int
    },
    "Video Likes Removed": {
        "title": "likes_removed",
        "data_type": int
    },
    "User Subscriptions Added": {
        "title": "subscribers_added",
        "data_type": int
    },
    "User Subscriptions Removed": {
        "title": "subscribers_removed",
        "data_type": int
    },
    "Average View Percentage": {
        "title": "avg_views_percent",
        "data_type": float
    },
    "Average Watch Time": {
        "title": "avg_watch_time",
        "data_type": float
    },
    "User Comments Added": {
        "title": "comments",
        "data_type": int
    }
}

In [26]:
# Analysis of video metrics
video_metrics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224 entries, 0 to 223
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Video                                 224 non-null    object 
 1   Video title                           223 non-null    object 
 2   Video pub­lish time                   223 non-null    object 
 3   Com­ments ad­ded                      224 non-null    int64  
 4   Shares                                224 non-null    int64  
 5   Dis­likes                             224 non-null    int64  
 6   Likes                                 224 non-null    int64  
 7   Sub­scribers lost                     224 non-null    int64  
 8   Sub­scribers gained                   224 non-null    int64  
 9   RPM (USD)                             224 non-null    float64
 10  CPM (USD)                             222 non-null    float64
 11  Av­er­age per­cent­

In [27]:
video_metrics.head(2)

,Video,Video title,Video pub­lish time,Com­ments ad­ded,Shares,Dis­likes,Likes,Sub­scribers lost,Sub­scribers gained,RPM (USD),CPM (USD),Av­er­age per­cent­age viewed (%),Av­er­age view dur­a­tion,Views,Watch time (hours),Sub­scribers,Your es­tim­ated rev­en­ue (USD),Im­pres­sions,Im­pres­sions click-through rate (%)
0,Total,NaN,NaN,14197,39640,3902,225021,45790,229241,5.276,11.990,26.61,0:03:25,5568487,317602.3536,183451,29068.652,100954064,3.16
1,4OZip0cgOho,How I Would Learn Data Science (If I Had to St...,"May 8, 2020",907,9583,942,46903,451,46904,6.353,12.835,36.65,0:03:09,1253559,65850.7042,46453,7959.533,26498799,3.14


In [32]:
VIDEO_METRICS_SCHEMA = {
    "Video": {
        "title": "video_id",
        "data_type": str
    },
    "Video title": {
        "title": "title",
        "data_type": str
    },
    "Video publish time": {
        "title": "publish_time",
        "data_type": "datetime"
    },
    "Comments added": {
        "title": "comments_added",
        "data_type": int
    },
    "Shares": {
        "title": "shares",
        "data_type": int
    },
    "Dislikes": {
        "title": "dislikes",
        "data_type": int
    },
    "Likes": {
        "title": "likes",
        "data_type": int
    },
    "Subscribers lost": {
        "title": "subscribers_lost",
        "data_type": int
    },
    "Subscribers gained": {
        "title": "subscribers_gained",
        "data_type": int
    },
    "RPM (USD)": {
        "title": "revenue_per_1000_usd",
        "data_type": float
    },
    "CPM (USD)": {
        "title": "cost_per_1000_usd",
        "data_type": float
    },
    "Average percentage viewed (%)": {
        "title": "avg_percent_viewed",
        "data_type": float
    },
    "Average view duration": {
        "title": "avg_view_duration",
        "data_type": "timedelta"
    },
    "Views": {
        "title": "views",
        "data_type": int
    },
    "Watch time (hours)": {
        "title": "watch_time_hr",
        "data_type": float
    },
    "Subscribers": {
        "title": "subscribers",
        "data_type": int
    },
    "Your estimated revenue (USD)": {
        "title": "estimated_revenue",
        "data_type": float
    },
    "Impressions": {
        "title": "impressions",
        "data_type": int
    },
    "Impressions click-through rate (%)": {
        "title": "impression_click_rate",
        "data_type": float
    }
}

### 3.1 Typecasting and 3.2 renaming data

In [17]:
'''
country_metrics = pd.read_csv(COUNTRY_SUB_METRICS)
video_metrics = pd.read_csv(VIDEO_METRICS)
comments = pd.read_csv(ALL_COMMENTS)
video_performance = pd.read_csv(VIDEO_PERFORMANCE)
'''

'\ncountry_metrics = pd.read_csv(COUNTRY_SUB_METRICS)\nvideo_metrics = pd.read_csv(VIDEO_METRICS)\ncomments = pd.read_csv(ALL_COMMENTS)\nvideo_performance = pd.read_csv(VIDEO_PERFORMANCE)\n'

In [50]:
# to time delta
# https://pandas.pydata.org/docs/reference/api/pandas.to_timedelta.html

def type_cast_column(col, schema):
    # search through schema to confirm title is present
    found_dtype = schema[col.name]
    if found_dtype == int:
        return col.astype(np.int32)
    elif found_dtype == float:
        return col.astype(np.float32)
    elif found_dtype == 'datetime':
        return pd.to_datetime(col)
    elif found_dtype == 'timedelta':
        # convert to timedelta seconds
        return pd.to_timedelta(col, unit='S')
    else:
        return col

In [55]:
def type_cast_df(df, schema):
    cpy = df.copy()
    # update column names as there are issues with unicode issues with title for video performance metrics
    new_col_names = [item['title'] for key, item in schema.items()]
    cpy.columns = new_col_names
    # create new dict from schema
    schema_upt = {item['title']:item['data_type'] for key, item in schema.items()}
    for col in cpy.columns:
        cpy[col] = type_cast_column(cpy[col], schema_upt)

    return cpy

In [98]:
video_performance_upt = type_cast_df(video_performance, VIDEO_PERFORMANCE_SCHEMA)
comments_upt = type_cast_df(comments, COMMENTS_SCHEMA)
country_metrics_upt = type_cast_df(country_metrics, COUNTRY_METRICS_SCHEMA)
video_metrics_upt = type_cast_df(video_metrics, VIDEO_METRICS_SCHEMA)

In [63]:
size_upt = calc_df_size(video_performance_upt, comments_upt, country_metrics_upt, video_metrics_upt)
print(f'Before: {size}, After: {size_upt}')
print(f'Decrease in size = {round((size - size_upt) / size * 100, 2)}')

Before: 19.38398, After: 12.60266
Decrease in size = 34.98


### 3.3 Sorting data

In [99]:
video_performance_upt.sort_values(by="date", inplace=True)
comments_upt.sort_values(by="date", inplace=True)
# country_metrics_upt.sort_values(by="date", inplace=True)
video_metrics_upt.sort_values(by='publish_time', inplace=True)

## 4. Conclusion

### 4.1 Observation of data

In [100]:
# video performance
video_performance_upt.describe(include='all')

,date,title,video_id,length_sec,thumbnail,views,likes_added,dislikes_added,likes_removed,subscribers_added,subscribers_removed,avg_views_percent,avg_watch_time,comments
count,111857,111857,111857,111857.000000,111857,111857.000000,111857.000000,111857.000000,111857.000000,111857.000000,111857.000000,110510.000000,110510.000000,111857.0
unique,1523,223,223,NaN,223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,2021-10-04 00:00:00,Predicting Crypto-Currency Price Using RNN lST...,qfRhKHV8-t4,NaN,https://i.ytimg.com/vi/qfRhKHV8-t4/hqdefault.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,202,1522,1522,NaN,1522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
first,2017-11-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,2022-01-19 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,874.615876,NaN,49.714287,2.095533,0.052424,0.086789,1.169180,0.026391,0.350455,216.766586,0.0
std,NaN,NaN,NaN,861.297667,NaN,316.557515,14.263695,1.852013,1.793532,12.109065,0.313908,0.232563,190.707611,0.0
min,NaN,NaN,NaN,47.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,NaN,NaN,NaN,375.000000,NaN,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.178010,117.379082,0.0


In [78]:
# comments 
comments_upt.describe(include='all')

,comment,comment_id,reply_count,like_count,date,video_id,user_id
count,10239,10240,10240.000000,10240.000000,10240,10240,10240
unique,10135,10240,NaN,NaN,10238,222,5875
top,I like it!,UgwPYqQkhITNHoyENH94AaABAg,NaN,NaN,2020-06-26 13:34:44+00:00,4OZip0cgOho,user_2385
freq,12,1,NaN,NaN,2,797,150
first,NaN,NaN,NaN,NaN,2017-12-03 18:52:35+00:00,NaN,NaN
last,NaN,NaN,NaN,NaN,2022-01-22 08:13:29+00:00,NaN,NaN
mean,NaN,NaN,1.337695,2.767578,NaN,NaN,NaN
std,NaN,NaN,1.096668,10.803318,NaN,NaN,NaN
min,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN
25%,NaN,NaN,1.000000,1.000000,NaN,NaN,NaN


In [101]:
# country metrics
country_metrics_upt.describe(include='all')

,title,video_id,length_sec,thumbnail,coutry,subscribed,views,likes_added,dislikes_added,likes_removed,subscription_added,subscription_removed,avg_views_percent,avg_watch_time,comments
count,55292,55292,55292.000000,55292,54906,55292,55292.000000,55292.000000,55292.000000,55292.000000,55292.000000,55292.000000,53854.000000,53854.000000,55292.0
unique,223,223,NaN,223,233,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,How I Would Learn Data Science (If I Had to St...,4OZip0cgOho,NaN,https://i.ytimg.com/vi/4OZip0cgOho/hqdefault.jpg,IN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,428,428,NaN,428,445,28588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,896.678073,NaN,NaN,NaN,100.573157,4.239311,0.106055,0.175577,2.365279,0.053389,0.347821,205.298340,0.0
std,NaN,NaN,927.380801,NaN,NaN,NaN,1704.966002,65.291451,2.507351,3.601171,65.889539,0.766945,0.259732,206.541626,0.0
min,NaN,NaN,47.000000,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,NaN,NaN,375.000000,NaN,NaN,NaN,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.145948,85.442894,0.0
50%,NaN,NaN,545.000000,NaN,NaN,NaN,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.320983,170.139725,0.0
75%,NaN,NaN,934.000000,NaN,NaN,NaN,27.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.486153,259.449951,0.0


In [84]:
country_metrics_upt[country_metrics_upt['views'] == 285593.000000]

,title,video_id,length_sec,thumbnail,coutry,subscribed,views,likes_added,dislikes_added,likes_removed,subscription_added,subscription_removed,avg_views_percent,avg_watch_time,comments
35823,How I Would Learn Data Science (If I Had to St...,4OZip0cgOho,516,https://i.ytimg.com/vi/4OZip0cgOho/hqdefault.jpg,US,False,285593,9165,399,327,0,0,0.394064,203.337006,0


In [103]:
# video_metrics
video_metrics_upt.describe(include='all')

,video_id,title,publish_time,comments_added,shares,dislikes,likes,subscribers_lost,subscribers_gained,revenue_per_1000_usd,cost_per_1000_usd,avg_percent_viewed,avg_view_duration,views,watch_time_hr,subscribers,estimated_revenue,impressions,impression_click_rate
count,223,223,223,223.000000,223.000000,223.000000,223.000000,223.000000,223.000000,223.000000,221.000000,223.000000,223,2.230000e+02,223.000000,223.000000,223.000000,2.230000e+02,223.000000
unique,223,223,222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,hOUrxTeBJQc,Find a Data Science Project With These 3 Techn...,2019-03-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
first,NaN,NaN,2017-06-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,2022-01-17 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,63.654709,177.753363,17.497758,1008.865471,13.260090,587.309417,4.438302,11.440302,34.120804,0 days 00:03:20.659192,2.496860e+04,1424.201782,574.049327,130.348633,4.526904e+05,3.083812
std,NaN,NaN,NaN,91.559620,733.331174,69.578517,3577.895609,33.242009,3302.316610,1.792296,3.342146,15.144405,0 days 00:01:34.649236,8.960956e+04,4804.354004,3270.679889,554.503662,1.864351e+06,1.674198
min,NaN,NaN,NaN,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,5.439000,5.230000,0 days 00:00:25,6.000000e+01,1.068400,-21.000000,0.000000,3.650000e+02,0.490000
25%,NaN,NaN,NaN,18.000000,19.000000,1.000000,163.000000,2.000000,27.000000,3.219500,9.377000,23.515000,0 days 00:02:24.500000,3.933000e+03,176.995552,19.500000,12.145500,9.894000e+04,1.920000


In [104]:
video_metrics_tot = video_metrics_upt.loc[0, :]
video_metrics_upt = video_metrics_upt.drop(0, axis=0).reset_index(drop=True)

### 4.2 Wrap up
1. Video metrics's column titles include unicode characters which are not read properly by pandas
    - function to update column data types considers only the updated titles from each individual dataframe's schema
2. Video metrics includes a row describing total of each column, the final total row is saved under the `video_metrics_tot` variable, and is dropped from our dataframe
3. There exists missing values in all of the data